In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
DATA_IN_PATH = './data_in/' 
TRAIN_CLEAN_DATA = 'train_clean.csv'
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [3]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
#min_df: ignore terms that have df strictly lower than the given threshold.
#analyzer=[word, char, char_wb] : feature should be made of word or character n-grams
#sublinear_tf: Apply sublinear tf scaling, i.e. replace tf with 1+log(tf).
#ngram_range: lower and upper boundary of the range of n-values for n-grams 
#max_features: build a vocabulary that only consider the top max_features ordered by TF 
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", 
            sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [8]:
print(X.shape)
#print(y.shape)
#X1 = vectorizer.fit(reviews)
#print(X1.vocabulary_)

(150000, 5000)


In [9]:
idf = vectorizer.idf_
features = vectorizer.get_feature_names()
#print(dict(zip(vectorizer.get_feature_names(), idf)))

In [10]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2
X_train, X_eval, y_train, y_eval = train_test_split(X, y, 
                test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [11]:
lgs = LogisticRegression(class_weight='balanced') 
lgs.fit(X_train, y_train) 

C:\Users\yshon\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
predicted = lgs.predict(X_eval)

In [13]:
print(predicted[277])
print(y_eval[277])
#print(predicted[1000])
#print(y_eval[1000])

0
0


In [14]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.807567


In [15]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [16]:
testDataVecs = vectorizer.transform(test_data['review'])

In [17]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 1 1 ... 0 0 0]


In [18]:
import csv
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/' 
test_file = DATA_IN_PATH + 'ratings_test.txt'
test_data = pd.read_csv(test_file, header=0, delimiter='\t', quoting=csv.QUOTE_NONE)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)